In [ ]:
pip install pyspark


In [ ]:
import os
from pyspark import SparkConf,SparkContext

from pyspark.sql import SparkSession
spark= SparkSession.builder.master("local").appName("rdd_demo").getOrCreate()

conf= SparkConf().setMaster("local").setAppName("Minitempreture")
sc= SparkContext.getOrCreate()

In [ ]:
spark

In [ ]:
flightdata=spark\
.read\
.option("header","true")\
.option("inferSchema","true")\
.csv("2015-summary.csv")


In [ ]:
print(flightdata)

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: int]


In [ ]:
flightdata

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: int]

In [ ]:
type(flightdata)

pyspark.sql.dataframe.DataFrame

In [ ]:
flightdata=spark\
.read\
.option("header","true")\
.option("inferSchema","true")\
.csv("2015-summary.csv")

flightdata=flightdata.toDF("dest","source","count").rdd
print(type(flightdata))
print(flightdata.take(1))
print(flightdata.count())

<class 'pyspark.rdd.RDD'>
[Row(dest='United States', source='Romania', count=15)]
256


In [ ]:
spath="2015-summary.csv"
sc_flightdata=spark.sparkContext.textFile(spath)
print(type(sc_flightdata))
print(sc_flightdata.take(2))

<class 'pyspark.rdd.RDD'>
['DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count', 'United States,Romania,15']


In [ ]:
import pandas as pd
spath="2015-summary.csv"
pd_flightdata=pd.read_csv(spath,header=0)
print(type(pd_flightdata))
print(pd_flightdata.head())
pd_flightdata=spark.createDataFrame(pd_flightdata)
print(type(pd_flightdata))
print(pd_flightdata.take(1))

<class 'pandas.core.frame.DataFrame'>
  DEST_COUNTRY_NAME ORIGIN_COUNTRY_NAME  count
0     United States             Romania     15
1     United States             Croatia      1
2     United States             Ireland    344
3             Egypt       United States     15
4     United States               India     62
<class 'pyspark.sql.dataframe.DataFrame'>
[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15)]


In [ ]:
mycollection="Spark The Definitive Guide : Big Data Processing Made Simple"\
.split(" ")
words=spark.sparkContext.parallelize(mycollection,2)
words.take(10)

['Spark',
 'The',
 'Definitive',
 'Guide',
 ':',
 'Big',
 'Data',
 'Processing',
 'Made',
 'Simple']

In [ ]:
mycollection="Spark The Definitive Guide : Big Data Processing Made Simple"\
.split(" ")
words=spark.sparkContext.parallelize(mycollection,2)
words.setName("mywords")
print(words.name())
print(type(words))
print(words.take(5))
words.getNumPartitions()

mywords
<class 'pyspark.rdd.RDD'>
['Spark', 'The', 'Definitive', 'Guide', ':']


2

In [ ]:
myRange=spark.range(1000000000000).toDF("number").rdd.map(lambda row:row[0])
print(type(myRange))
print(myRange.take(5))
myRange.getNumPartitions()


<class 'pyspark.rdd.PipelinedRDD'>
[0, 1, 2, 3, 4]


1

# filtering

In [ ]:
from datetime import date
def parseline(line):
    fields=line.split(",")
    date=fields[0]
    p_open=float(fields[1])
    p_close=float(fields[4])
    return (date,p_open,p_close)

In [ ]:
spath="RELIANCE.csv"
sdt=spark.sparkContext.textFile(spath)
sdt=sdt.map(parseline)
sdt

PythonRDD[325] at RDD at PythonRDD.scala:53

In [ ]:
spath="RELIANCE.csv"
o_sdt=spark.read.format("csv").option("header","true").option("inferSchema","true")\
.load(spath)
o_sdt=o_sdt.toDF("Date","Open","High","Low","Last","Close","Volume","Trunover").rdd\
.map(lambda row:(row[0],row[1],row[5]))
print(o_sdt.count())
print(type(o_sdt))
print(o_sdt.take(2))


o_sdt=o_sdt.filter(lambda row:row[2] > row[1])
print(o_sdt.take(5))
print(type(o_sdt))
print(o_sdt.count())

5
<class 'pyspark.rdd.PipelinedRDD'>
[(datetime.date(2022, 1, 1), 100, 10000), (datetime.date(2022, 1, 2), 200, 20000)]
[(datetime.date(2022, 1, 1), 100, 10000), (datetime.date(2022, 1, 2), 200, 20000), (datetime.date(2022, 1, 3), 300, 30000), (datetime.date(2022, 1, 4), 400, 40000), (datetime.date(2022, 1, 5), 500, 50000)]
<class 'pyspark.rdd.PipelinedRDD'>
5


In [ ]:
def Highclose(row):
    if row[2] > row[1]:
        return (row)

In [ ]:
o_sdt=spark.read.format("csv").option("header","true").load(spath)
o_sdt=o_sdt.toDF("Date","Open","High","Low","Last","Close","Volume","Trunover").rdd.map(lambda row:(row[0],row[1],row[5]))
print(o_sdt.count())
print(type(o_sdt))
print(o_sdt.take(2))

o_sdt=o_sdt.filter(lambda row:Highclose(row))
print(o_sdt.take(5))
print(type(o_sdt))
print(o_sdt.count())


5
<class 'pyspark.rdd.PipelinedRDD'>
[('2022-01-01', '100', '10000'), ('2022-01-02', '200', '20000')]
[('2022-01-01', '100', '10000'), ('2022-01-02', '200', '20000'), ('2022-01-03', '300', '30000'), ('2022-01-04', '400', '40000'), ('2022-01-05', '500', '50000')]
<class 'pyspark.rdd.PipelinedRDD'>
5


In [ ]:
def to_to_mill(row):
    return (row[0],row[1],round(row[3],0))

In [ ]:
spth="RELIANCE.csv"
o_sdt = spark.read.format("CSV").option("header","true").option("inferSchema", "true").load(spth)
o_sdt=o_sdt.toDF("Date","Open","High","Low","Last","Close","Volume","Turnover").rdd.map(lambda row: (row[0], row[1], row[5], row[7]))
print(o_sdt.take(2))
o_sdt=o_sdt.map(to_to_mill)
print(type(o_sdt))

[(datetime.date(2022, 1, 1), 100, 10000, None), (datetime.date(2022, 1, 2), 200, 20000, None)]
<class 'pyspark.rdd.PipelinedRDD'>


In [ ]:
def func(lines):
  lines=lines.lower()
  lines=lines.split(" ")
  return lines

In [ ]:
spath="shelock_holmes.txt"
input_file=sc.textFile(spath)


In [ ]:
print(input_file.take(1))

Py4JJavaError: An error occurred while calling o1264.partitions.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/content/shelock_holmes.txt
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:304)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:244)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:332)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:208)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:294)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:290)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:294)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:290)
	at org.apache.spark.api.java.JavaRDDLike.partitions(JavaRDDLike.scala:61)
	at or

In [ ]:
rdd1 = input_file.flatMap(Func)

NameError: name 'Func' is not defined

In [ ]:
rdd1.map(lambda x: (x,1)).take(5)

NameError: name 'rdd1' is not defined

In [ ]:
rdd2=rdd1 \
.map(lambda x: (x,1)) \
.groupByKey() \
.mapValues(sum) \
.map(lambda x: (x[1],x[0])) \
.sortByKey(False)


NameError: name 'rdd1' is not defined

In [ ]:
rdd2.take(5)

NameError: name 'rdd2' is not defined